In [1]:
import os
import shutil
import subprocess
from datetime import datetime
import warnings
from PIL import Image
import exifread

In [2]:
# set root folder:
folder = "/Users/en/Downloads/2018_bikealive"
# generate list of files with full paths:
flist = []
for (dirpath, dirnames, filenames) in os.walk(folder):
    # for each file:
    for f in filenames:
        fpath = os.path.join(dirpath, f)
        # ignore hidden files:
        if f[0] != '.':
            flist.append(fpath)

In [3]:
def getMDLSDatInfo(fileName):
    cmd = "mdls"\
          " -name kMDItemContentCreationDate" \
          " -name kMDItemContentModificationDate" \
          " -name kMDItemFSContentChangeDate" \
          " -name kMDItemFSCreationDate" \
          " '{:s}'".format(fileName)
    ret = subprocess.run(cmd,capture_output=True, shell=True)
    # check for errors:
    err = ret.stderr.decode('utf-8').split("\n")
    if not err[0] == '':
        raise Exception(err)
    # parse "ret":
    pOut = ret.stdout.decode('utf-8').split("\n")
    output = {}
    # get dates:
    output['created'] = datetime.strptime(pOut[0].split("=")[1], ' %Y-%m-%d %H:%M:%S %z')
    output['modified'] = datetime.strptime(pOut[1].split("=")[1], ' %Y-%m-%d %H:%M:%S %z')
    output['file_changed'] = datetime.strptime(pOut[2].split("=")[1], ' %Y-%m-%d %H:%M:%S %z')
    output['file_created'] = datetime.strptime(pOut[3].split("=")[1], ' %Y-%m-%d %H:%M:%S %z')
    
    return output

In [4]:
def getExifTags(filename=''):
    if os.path.isfile(filename):
        f = open(filename, 'rb')
        tags = exifread.process_file(f)
        outTags = {}
        for tag in tags.keys():
            if tag not in ('JPEGThumbnail', 'TIFFThumbnail', 'Filename', 'EXIF MakerNote'):
#                 print("Key: {:s}, value {:s}".format(tag, tags[tag].printable))
                outTags[tag] = tags[tag]
        f.close()
        return tags
    else:
        return []

In [5]:
## try for each file to read tags and get 'EXIF DateTimeOriginal'
fmap = {}
# set dst path:
_dstPath = "/Users/en/Downloads/2018_processed/"
for f in flist:
    try:
        # get path:
        _path = ('/').join(f.split('/')[:-1]) + '/'
        # get root file name:
        _file = f.split('/')[-1]
        # get extension
        _, _ext = os.path.splitext(_file)
        
        
        _tags = getExifTags(f)
        _date = _tags['EXIF DateTimeOriginal'].printable + " +00:00"
        d_created = datetime.strptime(_date, '%Y:%m:%d %H:%M:%S %z')
        d_created_str = d_created.strftime("%Y_%m_%d-%H_%M_%S%z")
        
        # src = f
        # dst = _dstPath + d_created_str + _ext
        _dst = _dstPath + d_created_str + _ext
#         print("{:s} ::::===> {:s}".format(f, _dst))
        # map src to dst
        fmap[f] = _dst
    except Exception as E:
        print(E)



'EXIF DateTimeOriginal'


In [6]:
# now check for duplicate targets:
dsts = [fmap[k] for k in fmap]

In [7]:
flist

['/Users/en/Downloads/2018_bikealive/6fc2c-8-18-037_30412806698_o.jpg',
 '/Users/en/Downloads/2018_bikealive/6fc2c-8-18-034_30412808338_o.jpg',
 '/Users/en/Downloads/2018_bikealive/6fc2c-8-18-320_42471522790_o.jpg',
 '/Users/en/Downloads/2018_bikealive/6fc2c-8-18-378_44231051732_o.jpg',
 '/Users/en/Downloads/2018_bikealive/6fc2c-8-18-207_43562342034_o.jpg',
 '/Users/en/Downloads/2018_bikealive/6fc2c-8-18-129_44279780241_o.jpg',
 '/Users/en/Downloads/2018_bikealive/6fc2c-8-18-164_44231208802_o.jpg',
 '/Users/en/Downloads/2018_bikealive/6fc2c-8-18-010_44231042982_o.jpg',
 '/Users/en/Downloads/2018_bikealive/6fc2c-8-18-295_29343673537_o.jpg',
 '/Users/en/Downloads/2018_bikealive/6fc2c-8-18-009_43562516664_o.jpg',
 '/Users/en/Downloads/2018_bikealive/6fc2c-8-18-259_44230956252_o.jpg',
 '/Users/en/Downloads/2018_bikealive/6fc2c-8-18-284_44231111282_o.jpg',
 '/Users/en/Downloads/2018_bikealive/6fc2c-8-18-266_43562131084_o.jpg',
 '/Users/en/Downloads/2018_bikealive/6fc2c-8-18-376_43562206684_

In [8]:
# do move...

for ff in fmap:
    src = ff
    dst = fmap[ff]
    # verify src exists
    if not os.path.isfile(src):
        raise Exception("source file {:s} doesnt exist!\n".format(src))
    # check if dst already exists..
    # check if dst file exists
    duplicate = os.path.isfile(dst)
    ii = 1
    #if dst file exists, add postfix to file with increment
    while (duplicate):
        warnings.warn("desination file {:s} EXISTS from source: {:s}!\n".format(dst, src))
        # split dst at extension and add postfix..
        _b, _bext = os.path.splitext(dst)
        dst = _b + "_{:03d}".format(ii) + _bext
        duplicate = os.path.isfile(dst)
        ii += 1
    # do move
    shutil.move(src,dst)

/Users/en/Git/temp/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: desination file /Users/en/Downloads/2018_processed/2018_08_15-10_27_55+0000.jpg EXISTS from source: /Users/en/Downloads/2018_bikealive/6fc2c-8-18-226_44230961302_o.jpg!

  from ipykernel import kernelapp as app
/Users/en/Git/temp/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: desination file /Users/en/Downloads/2018_processed/2018_08_18-10_36_54+0000.jpg EXISTS from source: /Users/en/Downloads/2018_bikealive/6fc2c-8-18-366_43373714385_o.jpg!

  from ipykernel import kernelapp as app
/Users/en/Git/temp/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: desination file /Users/en/Downloads/2018_processed/2018_08_14-10_16_40+0000.jpg EXISTS from source: /Users/en/Downloads/2018_bikealive/6fc2c-8-18-050_43373926405_o.jpg!

  from ipykernel import kernelapp as app
/Users/en/Git/temp/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning:

In [ ]:
fmap